This notebook will be mainly used for the IBM Applied Data Science Specialisation Capstone Project

In [1]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup

In [2]:
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


Part I - Segmenting and Clustering Neighborhoods in Toronto

Retreiving the URL of Wikipedia page that contains Toronto neighborhood data and creating a Beautiful soup object in order to scrape the Wikipedia page and wrangle the data.

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

#print(soup.prettify()) #Investigate the Beuatiful soup object

Wrangling and cleaning of Toronto neighborhood data in order to be read as a pandas data frame.
1) Postal code contains up to 3 characters and so is extracted using tablerow.p.text

2) Borough data is extracted by spliting the cell by '(' which is the begining of a Neigbourhood and so the first part [0] would be the Borough.

3) Neigbourhood data is the 2nd part but as more than one neighborhood can exist in one postal code area e.g. Harbourfront and Regent Park in M5A. These are combined into one row with the neighborhoods separated with a comma rather than a '/'.

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [5]:
table.findAll('td')[100].p.text[:3] #testing how to extract the data

'M2P'

In [6]:
(table.findAll('td')[100].span.text).split('(')

['North York', 'York Mills West)']

In [7]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Number of rows in dataframe.

In [8]:
df.shape

(103, 3)

Part I, Step 2

Obtaining the latitude and the longitude coordinates of each neighborhood. 

In [9]:
import geocoder # import geocoder

In [10]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format('M3A'))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

KeyboardInterrupt: 

In [11]:
#test for a more common example
while(lat_lng_coords is None):
    g = geocoder.google('Mountain View, CA')
    lat_lng_coords = g.latlng


KeyboardInterrupt: 

Geocoder did not work otherwise would have added latitude and longitude data as below.

In [12]:
a=np.empty(df.shape[0])
a[:]=np.nan
latitude=a.tolist()
longitude=a.tolist()

In [13]:
df['Latitude']=latitude
df['Longitude']=longitude

In [14]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN,NaN
4,M7A,Queen's Park,Ontario Provincial Government,NaN,NaN


In [15]:
df['PostalCode'][0]

'M3A'

In [16]:
for row in range(df.shape[0]):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(df['PostalCode'][row]))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    df['Latitude'][row]=latitude
    df['Longitude'][row]=longitude
    

KeyboardInterrupt: 

Instead will use csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [17]:
geo=pd.read_csv('Geospatial_Coordinates.csv')

In [18]:
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
geo.shape

(103, 3)

In [21]:
for row in range(df.shape[0]):
    df['Latitude'][row]=geo[geo['Postal Code']==df['PostalCode'][row]].Latitude
    df['Longitude'][row]=geo[geo['Postal Code']==df['PostalCode'][row]].Longitude

<ipython-input-21-8769348283df>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Latitude'][row]=geo[geo['Postal Code']==df['PostalCode'][row]].Latitude
<ipython-input-21-8769348283df>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Longitude'][row]=geo[geo['Postal Code']==df['PostalCode'][row]].Longitude


In [22]:
df.describe()

,Latitude,Longitude
count,103.000000,103.000000
mean,43.704608,-79.397153
std,0.052463,0.097146
min,43.602414,-79.615819
25%,43.660567,-79.464763
50%,43.696948,-79.388790
75%,43.745320,-79.340923
max,43.836125,-79.160497


In [23]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
